In [1]:
import sympy as sp
thp, thd, tp, td, lp, ld, alp, r, s = sp.symbols(r'\theta_p \theta_d t_p t_d l_p l_d \alpha r s', real = True)


In [2]:
## 1. distal actuation, no tendons on the proximal
kda = td / (ld * r)
kd1 = alp * kda
kd = 2 * (kda - kd1) * s / ld + kd1

kpa = tp / (lp * r)
kp1 = alp * kpa
kp = 2 * (kpa - kp1) * s / lp + kp1

듀얼벤딩 해석
- Distal 텐던 구동시 distal tendon에 의해 발생한 proximal angle 변화를 계산

- 동시에, proximal tendon의 길이 차분을 계산, 오프셋 부여

이후 목표 proximal tendon의 구동에 따른 proximal, distal angle의 차분을 계산

연립방정식으로 풀기

Distal tendon 구동에 의한 proximal tendon 길이 차분

힘줄의 길이 변화 $l_i - l'_i = r \kappa l_i$가 됨을 이용

In [25]:
# proximal tendon 각도 및 길이 차분
# thp_d = sp.integrate(kd, (s, 0, lp))
# dp = thp_d * r
# dd = sp.integrate(kd, (s, lp, ld)) * r
# display(thp_d)
# display(dp)
# dp = (ld - td) * (lp / ld)
# dd = (ld - td) - dp
dp = sp.integrate(r * kd, (s, 0, lp))
dd = sp.integrate(r * kd, (s, lp, ld))
display(sp.simplify(dp))
display(dd)

#check
display(sp.integrate(r * kd, (s, 0, ld)))
display(dp + dd)

l_p*t_d*(\alpha*l_d - l_p*(\alpha - 1))/l_d**2

-\alpha*l_p*t_d/l_d + t_d - l_p**2*(-\alpha*t_d + t_d)/l_d**2

t_d

t_d

2가지 경우 // 수정중
1. 동일방향으로 prox. tendon 추가구동시: l_p + tp가 구속조건이 됨
2. 반대방향으로 prox. tendon 추가구동시: l_p - tp가 구속조건이 됨

dist.tendon은 각 segment에서 미치는 영향력이 다름

prox seg에서는 prox tendon 구속에 의해 반지름 2r에 백본길이가 구속길이와 같아짐

dist seg에서는 prox seg에서 발생한 tendon 길이 차분에 의해 다소 변동이 발생

Case 해석
1. 동일방향
Prox. tendon만 구동했을 때 늘어난 쪽 tendon의 곡률:

$\kappa_{p,out} = \kappa_p \frac{l_p}{l_p+t_p}$

곡률반경의 닮음비를 이용하여 계산가능함

이 상태에서 디스탈을 구동

유효 백본 길이는 $l_d + t_p$가 된 상태

distal에 의한 곡률 형성: 평균곡률 먼저 구하기

$\kappa_{avg, d} = \begin{cases}
                    \frac{t_d}{2r(l_d+t_p)} (0 \le s \le l_p + t_p) \\
                    \frac{t_d}{r(l_d+t_p)}  (l_p + t_p \le s \le l_d + t_p)\\ 
                    \end{cases}$

이걸 가지고 prox 구속텐던의 곡률을 얻는다 >> 곡률 기반으로 구속안된쪽이 얼마나 수축한지를 구한다 >> 전체 길이에서 이거 빼고 디스털 구동 빼면 디스털 세그먼트 텐던 길이가 나옴 >> 디스털 길이에서 이걸 빼면 유효 디스털 TDL이 나온다

대충 $\cos(\phi_s)$ 이런식으로 일반화하면 될 듯 하다

In [4]:
# 일반화
p_s = sp.symbols('\phi_s')
dir = sp.cos(p_s)
tdconst = lp + tp * dir
kda_pr = td / (2 * r * (tdconst + ld - lp))
kd1_pr = alp * kda_pr
kd_pr = 2 * (kda_pr - kd1_pr) * s / (tdconst + ld - lp) + kd1_pr

kda = td / (ld * r)
kd1 = alp * kda
kd = 2 * (kda - kd1) * s / ld + kd1

kpa = tp / (lp * r)
kp1 = alp * kpa
kp = 2 * (kpa - kp1) * s / lp + kp1

ksum = kpa + kda
pang_int = sp.integrate(ksum, (s, 0, lp))
display(pang_int)

l_p*(t_p/(l_p*r) + t_d/(l_d*r))

위 각도는 무지성 곡률합성으로 얻은 것

이제 구속조건을 적용해보자

대충 계산때려보니까 이런식으로 나옴

$l_{cst} = l_p + \cos(\phi_s)(t_p - 2r\theta_p)$

$l_{p,new} = l_p + \cos(\phi_s)(t_p - r\theta_p) = l_p - \cos(\phi_s)\frac{l_p}{l_d}\frac{t_d}{r}$

prox.seg 영향을 무시한다면 tip angle은 이렇게 나올것이다:

$\theta_t = \frac{t_d + \cos(\phi_s)(t_p - r\theta_p)}{r} $

여기에서 prox. ang을 빼면 순수 dist.ang이 나올것이다

$\theta_d = \theta_t - \theta_p$

In [5]:
pang_eqn = sp.Eq(thp, (tp + sp.cos(p_s) * (td - r * thd)) / (2 * r))
tot_ang = (td + sp.cos(p_s) * (tp * r*thp)) / r
thd_eqn = sp.Eq(td, r * thd - sp.cos(p_s) * (tp - 2 * r * thp))

sol = sp.solve([pang_eqn, thd_eqn], (tp, td))
display(sol[tp])
display(sol[td])

2*\theta_p*r

\theta_d*r

순수기하로 prox ang 구하기

테스트

In [6]:
tp_eqn = sp.Eq(tp, 2 * r * thp + sp.cos(p_s) * (r * thd - td))
td_eqn = sp.Eq(td, sp.cos(p_s) * (2 * r * thp - tp) - r * thd)

sol = sp.solve([tp_eqn, td_eqn], (tp, td))
display(sol[tp])
display(sol[td])

(-2*\theta_d*r*cos(\phi_s) + 2*\theta_p*r*cos(\phi_s)**2 - 2*\theta_p*r)/(cos(\phi_s)**2 - 1)

(\theta_d*r*cos(\phi_s)**2 + \theta_d*r)/(cos(\phi_s)**2 - 1)

놀랍게도 내가 만들었던 식(ecavc)을 재구성

$l_{cst}$는 동일

distal에 의해 발생하는 길이변화를 적분으로 도출하여 길이를 얻다

근데 힘줄길이를 어케구함?

$\int (l - r\kappa(l))dl$ 하면 나온다

그러면 distal tendon에 의해 발생하는 곡률을 알아햐한다

이 때 반지름은 $2r$로 해야함(중심축에 대한 변화가 아님)

백본 전체길이 $l_d$ 또한 $l_{cst}$ 를 고려하여 변경해야한다

$l_{d,mod} = l_d - l_p + l_{cst} = l_d + \cos(\phi_s)t_p$

따라서

$\kappa_{avg, mod} = \frac{t_d}{2rl_{d,mod}}$

로 계산하면 곡률의 식

$\kappa_{d, mod}(s) = \frac{2(\kappa_{avg, mod} - \kappa_{1, mod})}{l_{d, mod}}(s) + \kappa_{1, mod}$

를 얻다

근데 상식적으로 곡률이 딱 반으로 나눠지는게 말이 되나?

닮음비로 다시 구해보자

$\kappa_{d, mod}(s) = \frac{\kappa_d(s)}{1 + r\kappa_d(s)}$

아니면, 그냥 디스탈 구동에 의해 축소된 길이를 먼저 구한 다음에 구속조건을 건다?

In [12]:
# 디스탈 구동 후 prox부분 텐던길이
with sp.assuming(sp.Q.nonzero(ld)):
    dp = sp.integrate(r * kd, (s, 0, lp))
display(dp.is_real)
display(sp.simplify(dp))

None

l_p*t_d*(\alpha*l_d - l_p*(\alpha - 1))/l_d**2

순차적으로 계산

prox tendon 걸려는 있는데 구동 없음(구속: $l_p$), 디스털은 구동 있음

이 때 prox에서 일어나는 일을 계산 << prox 각도의 변화

prox 각도 변화를 카운터치는 인풋 형성 + 디스탈 보정

원하는 prox 각도를 형성하는 tp 형성 + 디스탈 보정(디스탈은 유지한 채로 프록시말 구동)

prox angle을 0으로 만든 다음 prox 인풋을 주겠다

디스털에서는 실구동량(actual drive)과 유효구동량(effective drive) 간의 차이가 있다

프록시멀에서는 디스털 각도 형성에 의한 구동 오프셋이 발생(dp/2만큼 << 카운터 치면 반대쪽은 그만큼 늘어난다(디스털 텐던은 프록시멀에 길이구속을 주지 못함))

플롯해보니 대충 비슷한거같다

해석을 equiv. tdl 써서 하는게 보기좋을듯 하다

전에 했던 듀얼벤딩 실험 TDL하고 각도 포지션 값 가지고 견적내본 다음에 이후 작업 ㄱㄱ

전에 썼던 TDL을 쓰기 때문에 포워드로 개변 필요(thp_eqn, thd_eqn을 쓴다)

포지션의 경우 각도만 가지고 prox, dist 각각 오일러로 가정해서 하면될듯?

In [8]:
thd_eqn = sp.Eq(thd, (td - dp)/r)
thp_eqn = sp.Eq(thp, (tp + dp)/r)

sol = sp.solve([thd_eqn, thp_eqn], (tp, td))
display(sol[tp])
display(sol[td] + sol[tp])

(\alpha*\theta_d*l_d*l_p*r - \alpha*\theta_d*l_p**2*r + \alpha*\theta_p*l_d*l_p*r - \alpha*\theta_p*l_p**2*r + \theta_d*l_p**2*r - \theta_p*l_d**2*r + \theta_p*l_p**2*r)/(\alpha*l_d*l_p - \alpha*l_p**2 - l_d**2 + l_p**2)

-\theta_d*l_d**2*r/(\alpha*l_d*l_p - \alpha*l_p**2 - l_d**2 + l_p**2) + (\alpha*\theta_d*l_d*l_p*r - \alpha*\theta_d*l_p**2*r + \alpha*\theta_p*l_d*l_p*r - \alpha*\theta_p*l_p**2*r + \theta_d*l_p**2*r - \theta_p*l_d**2*r + \theta_p*l_p**2*r)/(\alpha*l_d*l_p - \alpha*l_p**2 - l_d**2 + l_p**2)

In [9]:
# 절댓값 테스트

x = sp.symbols('x', real = True)
tsteqn = sp.Eq(2, sp.Abs(x))
tstsol = sp.solve(tsteqn, x)
display(tstsol)

[-2, 2]

In [24]:
dp2 = sp.Rational(1,2) * dp

tp_eff = tp + dp2
td_eff_postd = td - dp2
td_eff_negtd = td + dp2

tpeqn = sp.Eq(tp_eff / r, thp)
tdposeqn = sp.Eq(td_eff_postd / r, thd)
tdnegeqn = sp.Eq(td_eff_negtd / r, thd)

possol = sp.solve([tpeqn, tdposeqn], (tp, td))
display(sp.cancel(possol[tp]))
display(sp.cancel(possol[td]))
display(sp.cancel(possol[tp] + possol[td]))

negsol = sp.solve([tpeqn, tdnegeqn], (tp, td))
display(negsol[tp])
display(negsol[td])
display(sp.cancel(negsol[tp] + negsol[td]))

(\alpha*\theta_d*l_d*l_p*r - \alpha*\theta_d*l_p**2*r + \alpha*\theta_p*l_d*l_p*r - \alpha*\theta_p*l_p**2*r + \theta_d*l_p**2*r - 2*\theta_p*l_d**2*r + \theta_p*l_p**2*r)/(\alpha*l_d*l_p - \alpha*l_p**2 - 2*l_d**2 + l_p**2)

-2*\theta_d*l_d**2*r/(\alpha*l_d*l_p - \alpha*l_p**2 - 2*l_d**2 + l_p**2)

\theta_d*r + \theta_p*r

(-\alpha*\theta_d*l_d*l_p*r + \alpha*\theta_d*l_p**2*r + \alpha*\theta_p*l_d*l_p*r - \alpha*\theta_p*l_p**2*r - \theta_d*l_p**2*r + 2*\theta_p*l_d**2*r + \theta_p*l_p**2*r)/(\alpha*l_d*l_p - \alpha*l_p**2 + 2*l_d**2 + l_p**2)

2*\theta_d*l_d**2*r/(\alpha*l_d*l_p - \alpha*l_p**2 + 2*l_d**2 + l_p**2)

(-\alpha*\theta_d*l_d*l_p*r + \alpha*\theta_d*l_p**2*r + \alpha*\theta_p*l_d*l_p*r - \alpha*\theta_p*l_p**2*r + 2*\theta_d*l_d**2*r - \theta_d*l_p**2*r + 2*\theta_p*l_d**2*r + \theta_p*l_p**2*r)/(\alpha*l_d*l_p - \alpha*l_p**2 + 2*l_d**2 + l_p**2)

In [31]:
# 그냥 tp, td >= 0 이라고 가정하고 phi_p phi_d 도입

p_p, p_d = sp.symbols(r'\phi_p \phi_d', real = True)
td_vct = td * sp.Matrix([sp.cos(p_d), sp.sin(p_d)])
tp_vct = tp * sp.Matrix([sp.cos(p_p), sp.sin(p_p)])
dp2_vct = dp2 * sp.Matrix([sp.cos(p_d), sp.sin(p_d)])

td_eff = td_vct - dp2_vct
tp_eff = tp_vct + dp2_vct

with sp.assuming(sp.Q.nonnegative(td),
                 sp.Q.nonnegative(tp),
                 sp.Q.nonnegative(lp),
                 sp.Q.nonnegative(ld)):
    display(sp.simplify(td_eff.norm()))

sqrt(Abs(t_d*(2*l_d**2 - l_p*(\alpha*l_d - l_p*(\alpha - 1)))*sin(\phi_d)/l_d**2)**2 + Abs(t_d*(2*l_d**2 - l_p*(\alpha*l_d - l_p*(\alpha - 1)))*cos(\phi_d)/l_d**2)**2)/2

모르겠다 그냥 식만 교체하고 값들은 그대로 쓰자

아래의 논지를 따라가면 된다

1. 일단 디스탈만 있다고 생각하고 구동, prox 부분의 길이 변화를 구한다
2. prox 카운터쳐서 직선으로 만든다(prox 보상구동)
3. 그 후 prox 부분을 원하는 만큼 구동, dist는 보상구동